<a href="https://colab.research.google.com/github/champtanapat/COS3109/blob/main/week_7_Naive_bayes_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to build and apply Naive Bayes classificator for spam filtering 


In [2]:
import pandas as pd

In [4]:
from google.colab import drive
ROOT = "/content/Gdrive"
drive.mount(ROOT)



Mounted at /content/Gdrive


Dataset from https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [6]:
sms_data = pd.read_csv('/content/Gdrive/MyDrive/Colab Notebooks/SMSSpamCollection', header=None, sep='\t', names=['Label', 'SMS'])


In [7]:
sms_data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
sms_data.shape

(5572, 2)

In [ ]:
sms_data.groupby('Label').count()

,SMS
Label,
ham,4825
spam,747


# Prepare data

In [80]:
sms_data_clean = sms_data.copy()

In [81]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()

<ipython-input-81-b182ea16a502>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()


In [82]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.lower()
print(sms_data_clean['SMS'])

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in 2 a wkly comp to win fa cup fina...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: SMS, Length: 5572, dtype: object


In [83]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.split()
#print(sms_data_clean['SMS'])

In [84]:
sms_data_clean['SMS'].head()

0    [go, until, jurong, point, crazy, available, o...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, don, t, think, he, goes, to, usf, he,...
Name: SMS, dtype: object

In [85]:
sms_data_clean['Label'].value_counts() / sms_data.shape[0] * 100


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

# Split to train and test data

In [86]:
train_data = sms_data_clean.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = sms_data_clean.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


In [87]:
train_data['Label'].value_counts() / train_data.shape[0] * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [88]:
train_data.shape

(4458, 2)

In [89]:
test_data['Label'].value_counts() / test_data.shape[0] * 100

ham     86.983842
spam    13.016158
Name: Label, dtype: float64

In [90]:
test_data.shape

(1114, 2)

In [91]:
test_data.head()

,Label,SMS
0,ham,"[aight, should, i, just, plan, to, come, up, l..."
1,ham,"[die, i, accidentally, deleted, e, msg, i, sup..."
2,spam,"[welcome, to, uk, mobile, date, this, msg, is,..."
3,ham,"[this, is, wishing, you, a, great, day, moji, ..."
4,ham,"[thanks, again, for, your, reply, today, when,..."


# Prepare vocabulary - the list of all the words from the dataset

In [92]:
vocabulary = list(set(train_data['SMS'].sum()))

#print(train_data['SMS'])
#vocabulary = list(set(train_data['SMS']))



In [93]:
vocabulary[11:20]

['lovely',
 'studentfinancial',
 'cysts',
 'meg',
 'work',
 'super',
 '02073162414',
 'sitter',
 'croydon']

In [94]:
len(vocabulary)

7783

# Calculate frequencies of the words for each message

In [95]:
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)

In [96]:
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [97]:
train_data.head()

,Label,SMS,weak,nasdaq,cat,unconscious,screamed,0808,cyclists,broke,...,marketing,character,scores,effects,classmates,virgins,dabooks,uncut,unsecured,milk
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculate values for the Bayes formula


In [98]:
alpha = 1

In [99]:
Nvoc = len(train_data.columns) - 3

In [100]:
Pspam = train_data['Label'].value_counts()['spam'] / train_data.shape[0]


In [101]:
Pham = train_data['Label'].value_counts()['spam'] / train_data.shape[0]


In [102]:
Nspam = train_data.loc[train_data['Label'] == 'spam', 'SMS'].apply(len).sum()

In [103]:
Nham = train_data.loc[train_data['Label'] == 'ham', 'SMS'].apply(len).sum()


In [104]:
def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1

In [105]:
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

# Prepare the classificator

In [106]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [107]:
classify('secret')


'ham'

In [108]:
classify(['secret', 'source', 'of', 'infinite', 'power'])


'spam'

In [109]:
classify(['free','win','welcom'])

'spam'

#Use test data

In [110]:
test_data['predicted'] = test_data['SMS'].apply(classify)

In [111]:
test_data.head()

,Label,SMS,predicted
0,ham,"[aight, should, i, just, plan, to, come, up, l...",ham
1,ham,"[die, i, accidentally, deleted, e, msg, i, sup...",ham
2,spam,"[welcome, to, uk, mobile, date, this, msg, is,...",spam
3,ham,"[this, is, wishing, you, a, great, day, moji, ...",ham
4,ham,"[thanks, again, for, your, reply, today, when,...",ham


In [112]:
correct = (test_data['predicted'] == test_data['Label']).sum() / test_data.shape[0] * 100

In [113]:
test_data.loc[test_data['predicted'] != test_data['Label']]


,Label,SMS,predicted
22,ham,"[erutupalam, thandiyachu]",spam
56,spam,"[money, i, have, won, wining, number, 946, wot...",ham
99,ham,"[gettin, rdy, to, ship, comp]",spam
142,ham,"[have, you, laid, your, airtel, line, to, rest]",spam
206,ham,"[no, yes, please, been, swimming]",spam
218,spam,"[hi, babe, its, chloe, how, r, u, i, was, smas...",ham
220,ham,"[wewa, is, 130, iriver, 255, all, 128, mb]",spam
244,ham,"[i, liked, the, new, mobile]",spam
245,ham,[anytime],spam
271,ham,"[i, career, tel, have, added, u, as, a, contac...",spam


In [114]:
correct


97.84560143626571